In [39]:
from google.colab import userdata
import os
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')

!kaggle competitions download -c store-sales-time-series-forecasting

!unzip -o store-sales-time-series-forecasting.zip -d store-sales

store-sales-time-series-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  store-sales-time-series-forecasting.zip
  inflating: store-sales/holidays_events.csv  
  inflating: store-sales/oil.csv     
  inflating: store-sales/sample_submission.csv  
  inflating: store-sales/stores.csv  
  inflating: store-sales/test.csv    
  inflating: store-sales/train.csv   
  inflating: store-sales/transactions.csv  


In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [41]:
DATA_PATH = 'store-sales/'
data_files = [f for f in os.listdir(DATA_PATH) if f.endswith('.csv')]

df = {}
for filename in data_files:
  df_name = filename.split('.')[0]
  df[df_name] = pd.read_csv(os.path.join(DATA_PATH, filename))
  print(f'{filename} loaded')

transactions.csv loaded
test.csv loaded
stores.csv loaded
sample_submission.csv loaded
holidays_events.csv loaded
train.csv loaded
oil.csv loaded


In [42]:
pd.options.display.float_format = '{:.4f}'.format
pd.options.display.max_columns = None

In [43]:
data = df['train'].copy()

data[['id', 'store_nbr']] = data[['id', 'store_nbr']].astype('category')
data['date'] = pd.to_datetime(data['date'])

start_date = data['date'].min()
end_date = data['date'].max()
all_dates = pd.date_range(start_date, end_date, freq='D')
missing_dates = all_dates.difference(data['date'])
print(missing_dates)

DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-12-25'], dtype='datetime64[ns]', freq=None)


In [44]:
all_stores = data['store_nbr'].unique()
all_families = data['family'].unique()
full_index = pd.MultiIndex.from_product([all_dates, all_stores, all_families], names=['date', 'store_nbr', 'family'])
full_data = pd.DataFrame(index=full_index).reset_index()
data = pd.merge(full_data, data, on=['date', 'store_nbr', 'family'], how='left')
data[['sales', 'onpromotion']] = data[['sales', 'onpromotion']].fillna(0)
data = data.sort_values(['date', 'store_nbr', 'family']).reset_index(drop=True)

## Features from analysis

In [45]:
data['year'] = data['date'].dt.year.astype('category')
data['month'] = data['date'].dt.month.astype('category')
data['day_of_week'] = data['date'].dt.dayofweek.astype('category')
last_day = data['date'].dt.is_month_end
payday_15 = data['date'].dt.day == 15
data['is_payday'] = (last_day | payday_15).astype('int')
data['is_weekend'] = data['day_of_week'].isin([5, 6]).astype('int')
is_dec_25 = ((data['date'].dt.month == 12) & (data['date'].dt.day == 25))
is_jan_1 = ((data['date'].dt.month == 1) & (data['date'].dt.day == 1))
data['is_christmas'] = (is_dec_25 | is_jan_1).astype('int')
data['was_earthquake'] = ((data['date'] >= pd.to_datetime('2016-04-16')) & (data['date'] <= pd.to_datetime('2016-04-21'))).astype('int')

In [46]:
data.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake
0,2013-01-01,1,AUTOMOTIVE,0,0.0000,0.0000,2013,1,1,0,0,1,0
1,2013-01-01,1,BABY CARE,1,0.0000,0.0000,2013,1,1,0,0,1,0
2,2013-01-01,1,BEAUTY,2,0.0000,0.0000,2013,1,1,0,0,1,0
3,2013-01-01,1,BEVERAGES,3,0.0000,0.0000,2013,1,1,0,0,1,0
4,2013-01-01,1,BOOKS,4,0.0000,0.0000,2013,1,1,0,0,1,0


## Lag features and rolling statistics

In [47]:
grouped_data = data.groupby(['store_nbr', 'family'])
for period in [1, 7, 15, 30, 365]:
  data[f'sales_lag_{period}'] = grouped_data['sales'].shift(period)

data[f'rolling_mean_30'] = grouped_data['sales'].transform(lambda x: x.rolling(window=30, min_periods=1).mean().shift(1))
data[f'rolling_std_30'] = grouped_data['sales'].transform(lambda x: x.rolling(window=30, min_periods=1).std().shift(1))
data[f'rolling_mean_15'] = grouped_data['sales'].transform(lambda x: x.rolling(window=15, min_periods=1).mean().shift(1))
data[f'rolling_std_15'] = grouped_data['sales'].transform(lambda x: x.rolling(window=15, min_periods=1).std().shift(1))
data[f'rolling_median_7'] = grouped_data['sales'].transform(lambda x: x.rolling(window=7, min_periods=1).median().shift(1))
data[f'rolling_mean_7'] = grouped_data['sales'].transform(lambda x: x.rolling(window=7, min_periods=1).mean().shift(1))

/tmp/ipython-input-622940319.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_data = data.groupby(['store_nbr', 'family'])


In [48]:
data[(data['store_nbr'] == 1) & (data['family'] == 'AUTOMOTIVE')][['sales', 'sales_lag_1', 'sales_lag_15', 'rolling_mean_7']][:16]

,sales,sales_lag_1,sales_lag_15,rolling_mean_7
0,0.0000,NaN,NaN,NaN
1782,2.0000,0.0000,NaN,0.0000
3564,3.0000,2.0000,NaN,1.0000
5346,3.0000,3.0000,NaN,1.6667
7128,5.0000,3.0000,NaN,2.0000
8910,2.0000,5.0000,NaN,2.6000
10692,0.0000,2.0000,NaN,2.5000
12474,2.0000,0.0000,NaN,2.1429
14256,2.0000,2.0000,NaN,2.4286
16038,2.0000,2.0000,NaN,2.4286


We can leave omissions, since some models can handle them well. In this case, the model understands that there are simply no statistics - this is better than filling in with zero. If the model does not know how to handle gaps, we can fill in the gaps with zero separately.

## Working with stores

In [49]:
df['stores'].head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [50]:
df['stores'].isnull().sum()

,0
store_nbr,0
city,0
state,0
type,0
cluster,0


In [51]:
print(df['stores']['city'].unique())
print(df['stores']['state'].unique())
print(df['stores']['type'].unique())
print(df['stores']['cluster'].unique())
print(df['stores'].dtypes)

['Quito' 'Santo Domingo' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil' 'Salinas' 'Daule' 'Babahoyo'
 'Quevedo' 'Playas' 'Libertad' 'Cuenca' 'Loja' 'Machala' 'Esmeraldas'
 'Manta' 'El Carmen']
['Pichincha' 'Santo Domingo de los Tsachilas' 'Cotopaxi' 'Chimborazo'
 'Imbabura' 'Bolivar' 'Pastaza' 'Tungurahua' 'Guayas' 'Santa Elena'
 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas' 'Manabi']
['D' 'B' 'C' 'E' 'A']
[13  8  9  4  6 15  7  3 12 16  1 10  2  5 11 14 17]
store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object


In [52]:
df['stores']['cluster'] = df['stores']['cluster'].astype('category')

in general, I think none of these features will interfere with the training of the model.

In [53]:
data = pd.merge(data, df['stores'], on='store_nbr', how='left')
data = data.rename(columns={'city': 'store_city', 'state': 'store_state', 'type': 'store_type', 'cluster': 'store_cluster'})
data.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,rolling_mean_30,rolling_std_30,rolling_mean_15,rolling_std_15,rolling_median_7,rolling_mean_7,store_city,store_state,store_type,store_cluster
0,2013-01-01,1,AUTOMOTIVE,0,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
1,2013-01-01,1,BABY CARE,1,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
2,2013-01-01,1,BEAUTY,2,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
3,2013-01-01,1,BEVERAGES,3,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13
4,2013-01-01,1,BOOKS,4,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13


## Holidays

In [54]:
holidays_events = df['holidays_events'].copy()

In [55]:
holidays_events.isnull().sum()

,0
date,0
type,0
locale,0
locale_name,0
description,0
transferred,0


In [56]:
holidays_events['date'] = pd.to_datetime(holidays_events['date'])

In [57]:
print(holidays_events['type'].unique(), '\n')
print(holidays_events['locale'].unique(), '\n')
print(holidays_events['locale_name'].unique(), '\n')
print(holidays_events['description'].unique())

['Holiday' 'Transfer' 'Additional' 'Bridge' 'Work Day' 'Event'] 

['Local' 'Regional' 'National'] 

['Manta' 'Cotopaxi' 'Cuenca' 'Libertad' 'Riobamba' 'Puyo' 'Guaranda'
 'Imbabura' 'Latacunga' 'Machala' 'Santo Domingo' 'El Carmen' 'Cayambe'
 'Esmeraldas' 'Ecuador' 'Ambato' 'Ibarra' 'Quevedo'
 'Santo Domingo de los Tsachilas' 'Santa Elena' 'Quito' 'Loja' 'Salinas'
 'Guayaquil'] 

['Fundacion de Manta' 'Provincializacion de Cotopaxi'
 'Fundacion de Cuenca' 'Cantonizacion de Libertad'
 'Cantonizacion de Riobamba' 'Cantonizacion del Puyo'
 'Cantonizacion de Guaranda' 'Provincializacion de Imbabura'
 'Cantonizacion de Latacunga' 'Fundacion de Machala'
 'Fundacion de Santo Domingo' 'Cantonizacion de El Carmen'
 'Cantonizacion de Cayambe' 'Fundacion de Esmeraldas'
 'Primer Grito de Independencia' 'Fundacion de Riobamba'
 'Fundacion de Ambato' 'Fundacion de Ibarra' 'Cantonizacion de Quevedo'
 'Independencia de Guayaquil' 'Traslado Independencia de Guayaquil'
 'Dia de Difuntos' 'Independencia d

In [58]:
holidays_events[holidays_events['locale'] == 'National']['locale_name'].unique()

array(['Ecuador'], dtype=object)

In [59]:
holidays_events[holidays_events['locale'] == 'National']['type'].unique()

array(['Holiday', 'Transfer', 'Additional', 'Bridge', 'Work Day', 'Event'],
      dtype=object)

In [60]:
holidays_events[holidays_events['locale'] == 'Regional']['locale_name'].unique()

array(['Cotopaxi', 'Imbabura', 'Santo Domingo de los Tsachilas',
       'Santa Elena'], dtype=object)

In [61]:
holidays_events[holidays_events['locale'] == 'Regional']['type'].unique()

array(['Holiday'], dtype=object)

1. There are holidays and events on a local (city), regional (states), and national (entire country) scale.

2. Transfer and Bridge should be replaced with Holiday, and Additional should be added to Event, thereby creating two binary signs - is_holiday and is_event.

In [62]:
holidays_events = holidays_events[
    (holidays_events['type'] != 'Work Day') | (holidays_events['transferred'] == True)
    ]
holidays_events['type'] = holidays_events['type'].replace({'Bridge': 'Holiday', 'Additional': 'Event', 'Transfer': 'Holiday'})
holidays_events['type'].unique()

array(['Holiday', 'Event'], dtype=object)

In [63]:
data = pd.merge(data, holidays_events, on='date', how='left')
data['is_holiday'] = (data['type'] == 'Holiday').astype('int')
data['is_event'] = (data['type'] == 'Event').astype('int')





data['store_in_holiday_area'] = 0

local_holiday_condition = (data['locale'] == 'Local') & (data['store_city'] == data['locale_name'])
data.loc[local_holiday_condition, 'store_in_holiday_area'] = 1

regional_holiday_condition = (data['locale'] == 'Regional') & (data['store_state'] == data['locale_name'])
data.loc[regional_holiday_condition, 'store_in_holiday_area'] = 1

national_holiday_condition = (data['locale'] == 'National')
data.loc[national_holiday_condition, 'store_in_holiday_area'] = 1

data = data.drop(columns=['type', 'locale_name', 'description', 'transferred']).rename(columns={'locale': 'holiday_locale'})
data.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,rolling_mean_30,rolling_std_30,rolling_mean_15,rolling_std_15,rolling_median_7,rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_holiday_area
0,2013-01-01,1,AUTOMOTIVE,0,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1
1,2013-01-01,1,BABY CARE,1,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1
2,2013-01-01,1,BEAUTY,2,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1
3,2013-01-01,1,BEVERAGES,3,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1
4,2013-01-01,1,BOOKS,4,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1


## Oil

In [64]:
oil_data = df['oil'].copy()
oil_data['date'] = pd.to_datetime(oil_data['date'])
missing_dates = all_dates.difference(oil_data['date'])
print(f'missing dates {len(missing_dates)}', missing_dates)
print('dcoilwtico missings', oil_data['dcoilwtico'].isnull().sum())

missing dates 482 DatetimeIndex(['2013-01-05', '2013-01-06', '2013-01-12', '2013-01-13',
               '2013-01-19', '2013-01-20', '2013-01-26', '2013-01-27',
               '2013-02-02', '2013-02-03',
               ...
               '2017-07-15', '2017-07-16', '2017-07-22', '2017-07-23',
               '2017-07-29', '2017-07-30', '2017-08-05', '2017-08-06',
               '2017-08-12', '2017-08-13'],
              dtype='datetime64[ns]', length=482, freq=None)
dcoilwtico missings 43


In [65]:
full_data = pd.DataFrame({'date': all_dates})
oil_data = pd.merge(full_data, oil_data, on='date', how='left').fillna(method='ffill').fillna(method='bfill').sort_values(by='date').reset_index(drop=True)
print('dcoilwtico missings', oil_data['dcoilwtico'].isnull().sum())

dcoilwtico missings 0


/tmp/ipython-input-787768181.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  oil_data = pd.merge(full_data, oil_data, on='date', how='left').fillna(method='ffill').fillna(method='bfill').sort_values(by='date').reset_index(drop=True)


In [66]:
oil_data.head()

,date,dcoilwtico
0,2013-01-01,93.1400
1,2013-01-02,93.1400
2,2013-01-03,92.9700
3,2013-01-04,93.1200
4,2013-01-05,93.1200


In [67]:
for period in [1, 7, 15, 30, 365]:
    oil_data[f'oil_lag_{period}'] = oil_data['dcoilwtico'].shift(period)


oil_data['oil_rolling_mean_30'] = oil_data['dcoilwtico'].rolling(window=30, min_periods=1).mean().shift(1)
oil_data['oil_rolling_std_30'] = oil_data['dcoilwtico'].rolling(window=30, min_periods=1).std().shift(1)
oil_data['oil_rolling_mean_15'] = oil_data['dcoilwtico'].rolling(window=15, min_periods=1).mean().shift(1)
oil_data['oil_rolling_std_15'] = oil_data['dcoilwtico'].rolling(window=15, min_periods=1).std().shift(1)
oil_data['oil_rolling_median_7'] = oil_data['dcoilwtico'].rolling(window=7, min_periods=1).median().shift(1)
oil_data['oil_rolling_mean_7'] = oil_data['dcoilwtico'].rolling(window=7, min_periods=1).mean().shift(1)

In [68]:
data = pd.merge(data, oil_data, on='date', how='left')

In [69]:
data[(data['store_nbr'] == 1) & (data['family'] == 'AUTOMOTIVE')]

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,rolling_mean_30,rolling_std_30,rolling_mean_15,rolling_std_15,rolling_median_7,rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_holiday_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7
0,2013-01-01,1,AUTOMOTIVE,0,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1782,2013-01-02,1,AUTOMOTIVE,1782,2.0000,0.0000,2013,1,2,0,0,0,0,0.0000,NaN,NaN,NaN,NaN,0.0000,NaN,0.0000,NaN,0.0000,0.0000,Quito,Pichincha,D,13,NaN,0,0,0,93.1400,93.1400,NaN,NaN,NaN,NaN,93.1400,NaN,93.1400,NaN,93.1400,93.1400
3564,2013-01-03,1,AUTOMOTIVE,3564,3.0000,0.0000,2013,1,3,0,0,0,0,2.0000,NaN,NaN,NaN,NaN,1.0000,1.4142,1.0000,1.4142,1.0000,1.0000,Quito,Pichincha,D,13,NaN,0,0,0,92.9700,93.1400,NaN,NaN,NaN,NaN,93.1400,0.0000,93.1400,0.0000,93.1400,93.1400
5346,2013-01-04,1,AUTOMOTIVE,5346,3.0000,0.0000,2013,1,4,0,0,0,0,3.0000,NaN,NaN,NaN,NaN,1.6667,1.5275,1.6667,1.5275,2.0000,1.6667,Quito,Pichincha,D,13,NaN,0,0,0,93.1200,92.9700,NaN,NaN,NaN,NaN,93.0833,0.0981,93.0833,0.0981,93.1400,93.0833
7128,2013-01-05,1,AUTOMOTIVE,7128,5.0000,0.0000,2013,1,5,0,1,0,0,3.0000,NaN,NaN,NaN,NaN,2.0000,1.4142,2.0000,1.4142,2.5000,2.0000,Quito,Pichincha,D,13,NaN,0,0,0,93.1200,93.1200,NaN,NaN,NaN,NaN,93.0925,0.0822,93.0925,0.0822,93.1300,93.0925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050784,2017-08-11,1,AUTOMOTIVE,2991978,1.0000,0.0000,2017,8,4,0,0,0,0,9.0000,8.0000,5.0000,7.0000,8.0000,5.3667,2.6844,5.5333,2.1668,7.0000,6.5714,Quito,Pichincha,D,13,National,1,0,1,48.8100,48.5400,49.5700,49.0500,45.4800,43.5100,47.9600,1.6291,49.4347,0.4019,49.5700,49.3257
3052566,2017-08-12,1,AUTOMOTIVE,2993760,6.0000,0.0000,2017,8,5,0,1,0,0,1.0000,5.0000,7.0000,9.0000,1.0000,5.1667,2.7803,5.2667,2.4631,6.0000,5.5714,Quito,Pichincha,D,13,NaN,0,0,0,48.8100,48.8100,49.5700,49.7200,46.0600,44.4700,48.0710,1.5665,49.4187,0.4225,49.3700,49.2171
3054348,2017-08-13,1,AUTOMOTIVE,2995542,1.0000,0.0000,2017,8,6,0,1,0,0,6.0000,6.0000,4.0000,4.0000,2.0000,5.0667,2.6901,5.2000,2.4260,6.0000,5.7143,Quito,Pichincha,D,13,NaN,0,0,0,48.8100,48.8100,49.5700,49.7200,46.5300,44.4700,48.1627,1.5247,49.3580,0.4411,49.0700,49.1086
3056130,2017-08-14,1,AUTOMOTIVE,2997324,1.0000,0.0000,2017,8,0,0,0,0,0,1.0000,7.0000,1.0000,6.0000,0.0000,4.9667,2.7852,5.0000,2.6458,6.0000,5.0000,Quito,Pichincha,D,13,NaN,0,0,0,47.5900,48.8100,49.3700,49.7200,46.5300,44.4700,48.2387,1.4971,49.2973,0.4502,48.8100,49.0000


## Transactions

In [71]:
transactions = df['transactions'].copy()
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


In [72]:
transactions['date'] = pd.to_datetime(transactions['date'])
missing_dates = all_dates.difference(transactions['date'])
print(f'missing dates {len(missing_dates)}', missing_dates)

missing dates 6 DatetimeIndex(['2013-12-25', '2014-12-25', '2015-12-25', '2016-01-01',
               '2016-01-03', '2016-12-25'],
              dtype='datetime64[ns]', freq=None)


In [76]:
full_index = pd.MultiIndex.from_product([all_dates, all_stores], names=['date', 'store_nbr'])
full_data = pd.DataFrame(index=full_index).reset_index()
transactions = pd.merge(full_data, transactions, on=['date', 'store_nbr'], how='left')
transactions['transactions'] = transactions['transactions'].fillna(0)
transactions = transactions.sort_values(['date', 'store_nbr']).reset_index(drop=True)
transactions.head()

,date,store_nbr,transactions
0,2013-01-01,1,0.0000
1,2013-01-01,2,0.0000
2,2013-01-01,3,0.0000
3,2013-01-01,4,0.0000
4,2013-01-01,5,0.0000


In [79]:
len(transactions) * 33

3008016

In [80]:
data = pd.merge(data, transactions, on=['date', 'store_nbr'], how='left')
data.head()

,date,store_nbr,family,id,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,rolling_mean_30,rolling_std_30,rolling_mean_15,rolling_std_15,rolling_median_7,rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_holiday_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
0,2013-01-01,1,AUTOMOTIVE,0,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,2013-01-01,1,BABY CARE,1,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
2,2013-01-01,1,BEAUTY,2,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
3,2013-01-01,1,BEVERAGES,3,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,2013-01-01,1,BOOKS,4,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000


# Save

In [81]:
data = data.drop(columns=['id', 'store_nbr'])
data.head()

,date,family,sales,onpromotion,year,month,day_of_week,is_payday,is_weekend,is_christmas,was_earthquake,sales_lag_1,sales_lag_7,sales_lag_15,sales_lag_30,sales_lag_365,rolling_mean_30,rolling_std_30,rolling_mean_15,rolling_std_15,rolling_median_7,rolling_mean_7,store_city,store_state,store_type,store_cluster,holiday_locale,is_holiday,is_event,store_in_holiday_area,dcoilwtico,oil_lag_1,oil_lag_7,oil_lag_15,oil_lag_30,oil_lag_365,oil_rolling_mean_30,oil_rolling_std_30,oil_rolling_mean_15,oil_rolling_std_15,oil_rolling_median_7,oil_rolling_mean_7,transactions
0,2013-01-01,AUTOMOTIVE,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
1,2013-01-01,BABY CARE,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
2,2013-01-01,BEAUTY,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
3,2013-01-01,BEVERAGES,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000
4,2013-01-01,BOOKS,0.0000,0.0000,2013,1,1,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Quito,Pichincha,D,13,National,1,0,1,93.1400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0000


In [85]:
data.to_csv('drive/MyDrive/data/store_sales_ecuador/data.csv', index=False)